# Tutorial 4 — FIFO Depth Optimization

This notebook demonstrates how to:
1. Create a simple U-Net model and convert it with the **VitisUnified** backend
2. Enable the `vitisunified:fifo_depth_optimization` flow
3. Verify that the optimization produces a `fifo_depths.json` file

## 1. Imports & Environment Setup

In [ ]:
import os
import json
from pathlib import Path

import numpy as np
from tensorflow.keras.layers import (
    Concatenate, Conv2D, Input, MaxPooling2D, UpSampling2D,
)
from tensorflow.keras.models import Model, load_model

import hls4ml
import hls4ml.model

In [ ]:
test_root_path = Path(".").resolve()

os.environ["XILINX_VITIS"] = "/tools/Xilinx/Vitis/2023.2"
os.environ["PATH"] = os.environ["XILINX_VITIS"] + "/bin:" + os.environ["PATH"]

XPFM_PATH_CUSTOM = (
    "/tools/Xilinx/Vitis/2023.2/base_platforms/"
    "xilinx_zcu102_base_202320_1/xilinx_zcu102_base_202320_1.xpfm"
)
XPFM_PATH_VENDOR = XPFM_PATH_CUSTOM

## 2. Helper Functions

In [ ]:
def create_io_file_dir():
    os.makedirs(test_root_path / "input_file", exist_ok=True)
    os.makedirs(test_root_path / "output_file", exist_ok=True)

def create_simple_testcase(inputShape=(4, 4, 1), fileName="inputX.npy"):
    n_in = np.random.rand(*inputShape).astype(np.float32)
    os.makedirs(test_root_path / "input_file", exist_ok=True)
    np.save(test_root_path / "input_file" / fileName, n_in)

def create_simple_unet(input_shape=(4, 4, 1), modelName="simpleSkip.keras"):
    inputs = Input(input_shape)
    c1 = Conv2D(2, (3, 3), activation="relu", padding="same")(inputs)
    p1 = MaxPooling2D((2, 2))(c1)
    bn = Conv2D(4, (3, 3), activation="relu", padding="same")(p1)
    u1 = UpSampling2D((2, 2))(bn)
    concat1 = Concatenate()([u1, c1])
    c2 = Conv2D(2, (3, 3), activation="relu", padding="same")(concat1)
    outputs = Conv2D(1, (1, 1), activation="sigmoid")(c2)
    model = Model(inputs, outputs)
    model.compile(optimizer="adam", loss="binary_crossentropy")
    model.save(test_root_path / "input_file" / modelName)
    return model

def gen_prj_dir(backend, io_type, strategy, granularity, prefix, axi_mode):
    return str(
        test_root_path
        / f"hls4mlprj_{prefix}_{backend}_{strategy}_{io_type}_{granularity}_{axi_mode}"
    )

def create_hls_model(model, config, backend, io_type, strategy, granularity, prefix, axi_mode):
    output_dir = gen_prj_dir(backend, io_type, strategy, granularity, prefix, axi_mode)
    hls_model = hls4ml.converters.convert_from_keras_model(
        model, hls_config=config, output_dir=output_dir, backend=backend,
        io_type=io_type, board="zcu102", part="xczu9eg-ffvb1156-2-e",
        clock_period="10ns", input_type="float", output_type="float",
        xpfmPath=XPFM_PATH_VENDOR if axi_mode == "axim" else XPFM_PATH_CUSTOM,
        axi_mode=axi_mode,
    )
    hls_model.compile()
    return hls_model

def create_hls_model4_cosim(
    model, config, backend, io_type, strategy, granularity,
    input_data_tb, output_data_tb, prefix, axi_mode,
):
    output_dir = gen_prj_dir(backend, io_type, strategy, granularity, prefix, axi_mode)
    hls_model = hls4ml.converters.convert_from_keras_model(
        model, hls_config=config, output_dir=output_dir, backend=backend,
        io_type=io_type, board="zcu102", part="xczu9eg-ffvb1156-2-e",
        clock_period="10ns", input_type="float", output_type="float",
        input_data_tb=input_data_tb, output_data_tb=output_data_tb,
        axi_mode=axi_mode,
    )
    hls_model.compile()
    return hls_model

## 3. Parameters

In [ ]:
io_type     = "io_stream"
strategy    = "latency"
granularity = "name"
amt_query   = 10
axi_mode    = "axim"        # change to "axis" to test AXI-Stream mode

## 4. Prepare Data & Model

In [ ]:
create_io_file_dir()

create_simple_testcase(inputShape=(amt_query, 4, 4, 1), fileName="inputFifoDepth.npy")
input_data = np.load(test_root_path / "input_file" / "inputFifoDepth.npy")
print(f"Input shape: {input_data.shape}")

model_name = "simpleSkipFifoDepth.keras"
create_simple_unet(modelName=model_name)
model = load_model(test_root_path / "input_file" / model_name)
model.summary()

## 5. Generate Reference Predictions

In [ ]:
config = hls4ml.utils.config_from_keras_model(model, granularity=granularity)

vitis_unified_model = create_hls_model(
    model, config, "VitisUnified", io_type, strategy, granularity, "fifodepth", axi_mode
)
y_hls4ml_unified = vitis_unified_model.predict(input_data)
np.save(test_root_path / "output_file" / "outputFifoDepth.npy", y_hls4ml_unified)
print(f"Reference predictions saved — shape: {np.array(y_hls4ml_unified).shape}")

## 6. Enable FIFO Depth Optimization & Build

In [ ]:
input_data_tb  = str(test_root_path / "input_file"  / "inputFifoDepth.npy")
output_data_tb = str(test_root_path / "output_file" / "outputFifoDepth.npy")

# Enable the FIFO depth optimization flow
config["Flows"] = ["vitisunified:fifo_depth_optimization"]

vitis_unified_model_fifo = create_hls_model4_cosim(
    model, config, "VitisUnified", io_type, strategy, granularity,
    input_data_tb, output_data_tb, "fifodepth", axi_mode,
)
vitis_unified_model_fifo.compile()

## 7. Verify FIFO Depths File

In [ ]:
prj_dir = gen_prj_dir("VitisUnified", io_type, strategy, granularity, "fifodepth", axi_mode)
fifodepth_path = prj_dir + "/fifo_depths.json"

assert os.path.exists(fifodepth_path), f"fifo_depths.json not found at {fifodepth_path}"

with open(fifodepth_path) as f:
    fifo_depths = json.load(f)

print("FIFO depths:")
for name, depth in fifo_depths.items():
    print(f"  {name}: {depth}")

print("\n✅ FIFO depth optimization completed successfully.")